In [1]:
import requests
url = 'https://worldcup.sfg.io/matches'
r = requests.get(url)
all_games = r.json()

In [2]:
from datetime import datetime as dt
from sqlalchemy import Column, Integer, String, ForeignKey, DateTime, create_engine
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker, relationship, backref

Base = declarative_base()

class Game(Base):
    __tablename__ = 'games'
    id = Column(Integer, primary_key=True)
    venue = Column(String)
    winner = Column(String)
    statistics = relationship('Statistics', back_populates="game")
    teams = relationship(
        'Team',
        secondary='statistics',
        back_populates='games'
    )

class Team(Base):
    __tablename__ = 'teams'
    id = Column(Integer, primary_key=True)
    country = Column(String)
    statistics = relationship('Statistics', back_populates="team")
    games = relationship(
        'Game',
        secondary='statistics',
        back_populates='teams'
    )

class Statistics(Base):
    __tablename__ = 'statistics'
    id = Column(Integer, primary_key=True)
    name = Column(String)
    game_id = Column(Integer, ForeignKey('games.id'))
    team_id = Column(Integer, ForeignKey('teams.id'))
    goals = Column(Integer)
    game = relationship('Game', back_populates='statistics')
    team = relationship('Team', back_populates='statistics')
    ball_possession=Column(Integer)
    distance_covered=Column(Integer)
    on_target=Column(Integer)
    pass_accuracy=Column(Integer)

engine = create_engine('sqlite:///worldcup.db')
Base.metadata.create_all(engine)
Session = sessionmaker(bind=engine)
session = Session()

In [3]:
all_games_tuples= [(game['venue'], game['home_team_country'], game['away_team_country'], game['winner']) for game in all_games]

In [4]:
h_teams = set([t[1] for t in all_games_tuples])
a_teams = set([t[2] for t in all_games_tuples])
teams = h_teams.union(a_teams)

In [5]:
team_objects = [Team(country = t) for t in teams]
len(team_objects)

32

In [6]:
session.add_all(team_objects)
session.commit()

In [7]:
all_game_objects = []
all_stats = []
for game in all_games:
    h_team = session.query(Team).filter(Team.country == game['home_team_statistics']['country']).one()
    a_team = session.query(Team).filter(Team.country == game['away_team_statistics']['country']).one()
    game_object = Game(venue= game['venue'],
                       winner = game['winner'])
    all_game_objects.append(game_object)


    h_name = game['home_team']['country']
    a_name = game['away_team']['country']
    h_goals = game['home_team']['goals']
    h_ball_possession = game['home_team_statistics']['ball_possession']
    a_goals = game['away_team']['goals']
    a_ball_possession = game['away_team_statistics']['ball_possession']
    h_distance_covered = game['home_team_statistics']['distance_covered']
    a_distance_covered = game['away_team_statistics']['distance_covered']
    h_pass_accuracy = game['home_team_statistics']['pass_accuracy']
    a_pass_accuracy = game['away_team_statistics']['pass_accuracy']
    h_on_target = game['home_team_statistics']['on_target']
    a_on_target = game['away_team_statistics']['on_target']


    h_stats = Statistics(game = game_object,
                         name= h_name,
                         team = h_team,
                        ball_possession =h_ball_possession,
                        goals = h_goals,
                        distance_covered = h_distance_covered,
                        pass_accuracy = h_pass_accuracy,
                        on_target = h_on_target)
    a_stats = Statistics(game = game_object,
                         name= a_name,
                        team = a_team,
                        ball_possession =a_ball_possession,
                        goals = a_goals,
                        distance_covered = a_distance_covered,
                        pass_accuracy = a_pass_accuracy,
                        on_target = a_on_target)
    all_stats.append(h_stats)
    all_stats.append(a_stats)


In [8]:
len(all_game_objects)

64

In [9]:
len(all_stats)

128

In [10]:
session.add_all(all_game_objects)
session.add_all(all_stats)
session.commit()

In [11]:
#all statistics objects where russia played
session.query(Statistics).join(Team).filter(Team.country == 'Russia').all()

In [12]:
#first statistics objects where russia played
first_russia_stat = session.query(Statistics).join(Team).filter(Team.country == 'Russia').all()[0]

In [13]:
first_russia_stat.ball_possession

40

In [14]:
from sqlalchemy.sql import func

In [15]:
def team_with_highest_ball_poss():
    max_bp= session.query(Statistics,func.max(Statistics.ball_possession)).join(Team).first()
    return max_bp.Statistics.name
team_with_highest_ball_poss()

'Spain'

In [16]:
def team_with_longest_dist_covered():
    max_dc= session.query(Statistics,func.max(Statistics.distance_covered)).join(Team).first()
    return max_dc.Statistics.name
team_with_longest_dist_covered()

'Russia'

In [17]:
def team_with_shortest_dist_covered():
    min_dc= session.query(Statistics,func.min(Statistics.distance_covered)).join(Team).first()
    return min_dc.Statistics.name
team_with_shortest_dist_covered()

'Poland'

In [18]:
def team_with_lowest_ball_poss():
    min_bp= session.query(Statistics,func.min(Statistics.ball_possession)).join(Team).first()
    return min_bp.Statistics.name
team_with_lowest_ball_poss()

'Russia'

In [19]:
def count_stat_goals(session):
    count_statgoals = session.query(func.count(Statistics.goals)).one()
    return count_statgoals

count_stat_goals(session)

(128)

In [20]:
def return_country_and_its_total_goals():
    return session.query(Team.country, func.sum(Statistics.goals)).join(Statistics).group_by(Team.country).all()

return_country_and_its_total_goals()

[('Argentina', 6),
 ('Australia', 2),
 ('Belgium', 16),
 ('Brazil', 8),
 ('Colombia', 6),
 ('Costa Rica', 2),
 ('Croatia', 14),
 ('Denmark', 3),
 ('Egypt', 2),
 ('England', 12),
 ('France', 14),
 ('Germany', 2),
 ('Iceland', 2),
 ('Iran', 2),
 ('Japan', 6),
 ('Korea Republic', 3),
 ('Mexico', 3),
 ('Morocco', 2),
 ('Nigeria', 3),
 ('Panama', 2),
 ('Peru', 2),
 ('Poland', 2),
 ('Portugal', 6),
 ('Russia', 11),
 ('Saudi Arabia', 2),
 ('Senegal', 4),
 ('Serbia', 2),
 ('Spain', 7),
 ('Sweden', 6),
 ('Switzerland', 5),
 ('Tunisia', 5),
 ('Uruguay', 7)]

In [22]:
def average_dc_vs_ot_per_game():
    return session.query(Team.country, func.avg(Statistics.distance_covered), func.avg(Statistics.on_target)).join(Statistics).group_by(Team.country).order_by(func.avg(Statistics.distance_covered).asc()).all()

average_dc_vs_ot_per_game()

[('Mexico', 95.25, 3.25),
 ('Panama', 96.33333333333333, 2.6666666666666665),
 ('Poland', 98.33333333333333, 2.6666666666666665),
 ('Japan', 99.0, 4.0),
 ('Nigeria', 99.0, 3.0),
 ('Argentina', 99.5, 4.5),
 ('Iran', 99.66666666666667, 1.3333333333333333),
 ('France', 99.85714285714286, 4.142857142857143),
 ('Portugal', 100.0, 3.25),
 ('Brazil', 100.6, 7.6),
 ('Senegal', 101.33333333333333, 4.0),
 ('Peru', 101.66666666666667, 3.6666666666666665),
 ('Saudi Arabia', 102.0, 3.3333333333333335),
 ('Tunisia', 103.33333333333333, 4.333333333333333),
 ('Uruguay', 103.4, 4.4),
 ('Belgium', 103.85714285714286, 5.714285714285714),
 ('Iceland', 104.0, 4.0),
 ('Sweden', 104.2, 4.0),
 ('Korea Republic', 105.66666666666667, 3.6666666666666665),
 ('Switzerland', 105.75, 3.5),
 ('Costa Rica', 106.0, 3.0),
 ('Morocco', 106.33333333333333, 3.3333333333333335),
 ('Colombia', 108.75, 3.0),
 ('Egypt', 109.33333333333333, 1.6666666666666667),
 ('Australia', 109.66666666666667, 2.6666666666666665),
 ('Germany'

In [23]:
def average_goals_vs_dc_per_game():
    return session.query(Team.country, func.avg(Statistics.goals), func.avg(Statistics.distance_covered)).join(Statistics).group_by(Team.country).order_by(func.avg(Statistics.goals).asc()).all()

average_goals_vs_dc_per_game()

[('Australia', 0.6666666666666666, 109.66666666666667),
 ('Costa Rica', 0.6666666666666666, 106.0),
 ('Egypt', 0.6666666666666666, 109.33333333333333),
 ('Germany', 0.6666666666666666, 111.33333333333333),
 ('Iceland', 0.6666666666666666, 104.0),
 ('Iran', 0.6666666666666666, 99.66666666666667),
 ('Morocco', 0.6666666666666666, 106.33333333333333),
 ('Panama', 0.6666666666666666, 96.33333333333333),
 ('Peru', 0.6666666666666666, 101.66666666666667),
 ('Poland', 0.6666666666666666, 98.33333333333333),
 ('Saudi Arabia', 0.6666666666666666, 102.0),
 ('Serbia', 0.6666666666666666, 112.0),
 ('Denmark', 0.75, 115.5),
 ('Mexico', 0.75, 95.25),
 ('Korea Republic', 1.0, 105.66666666666667),
 ('Nigeria', 1.0, 99.0),
 ('Sweden', 1.2, 104.2),
 ('Switzerland', 1.25, 105.75),
 ('Senegal', 1.3333333333333333, 101.33333333333333),
 ('Uruguay', 1.4, 103.4),
 ('Argentina', 1.5, 99.5),
 ('Colombia', 1.5, 108.75),
 ('Japan', 1.5, 99.0),
 ('Portugal', 1.5, 100.0),
 ('Brazil', 1.6, 100.6),
 ('Tunisia', 1.66

In [24]:
def average_goals_vs_bp_per_game():
    return session.query(Team.country, func.avg(Statistics.goals), func.avg(Statistics.ball_possession)).join(Statistics).group_by(Team.country).order_by(func.avg(Statistics.goals).asc()).all()

average_goals_vs_bp_per_game()

[('Australia', 0.6666666666666666, 51.333333333333336),
 ('Costa Rica', 0.6666666666666666, 40.333333333333336),
 ('Egypt', 0.6666666666666666, 45.0),
 ('Germany', 0.6666666666666666, 66.66666666666667),
 ('Iceland', 0.6666666666666666, 37.333333333333336),
 ('Iran', 0.6666666666666666, 32.666666666666664),
 ('Morocco', 0.6666666666666666, 49.666666666666664),
 ('Panama', 0.6666666666666666, 39.0),
 ('Peru', 0.6666666666666666, 51.333333333333336),
 ('Poland', 0.6666666666666666, 50.0),
 ('Saudi Arabia', 0.6666666666666666, 58.0),
 ('Serbia', 0.6666666666666666, 45.0),
 ('Denmark', 0.75, 45.5),
 ('Mexico', 0.75, 54.5),
 ('Korea Republic', 1.0, 40.666666666666664),
 ('Nigeria', 1.0, 45.666666666666664),
 ('Sweden', 1.2, 38.6),
 ('Switzerland', 1.25, 57.75),
 ('Senegal', 1.3333333333333333, 44.666666666666664),
 ('Uruguay', 1.4, 48.2),
 ('Argentina', 1.5, 63.75),
 ('Colombia', 1.5, 49.75),
 ('Japan', 1.5, 52.25),
 ('Portugal', 1.5, 54.0),
 ('Brazil', 1.6, 56.0),
 ('Tunisia', 1.6666666666

In [26]:
def average_goals_vs_pa_per_game():
    return session.query(Team.country, func.avg(Statistics.goals), func.avg(Statistics.pass_accuracy)).join(Statistics).group_by(Team.country).order_by(func.avg(Statistics.goals).asc()).all()

average_goals_vs_pa_per_game()

[('Australia', 0.6666666666666666, 84.33333333333333),
 ('Costa Rica', 0.6666666666666666, 76.0),
 ('Egypt', 0.6666666666666666, 80.33333333333333),
 ('Germany', 0.6666666666666666, 87.33333333333333),
 ('Iceland', 0.6666666666666666, 68.66666666666667),
 ('Iran', 0.6666666666666666, 68.0),
 ('Morocco', 0.6666666666666666, 82.33333333333333),
 ('Panama', 0.6666666666666666, 82.0),
 ('Peru', 0.6666666666666666, 81.0),
 ('Poland', 0.6666666666666666, 82.0),
 ('Saudi Arabia', 0.6666666666666666, 87.33333333333333),
 ('Serbia', 0.6666666666666666, 78.33333333333333),
 ('Denmark', 0.75, 80.0),
 ('Mexico', 0.75, 83.0),
 ('Korea Republic', 1.0, 76.33333333333333),
 ('Nigeria', 1.0, 78.66666666666667),
 ('Sweden', 1.2, 73.2),
 ('Switzerland', 1.25, 84.25),
 ('Senegal', 1.3333333333333333, 78.0),
 ('Uruguay', 1.4, 78.6),
 ('Argentina', 1.5, 85.25),
 ('Colombia', 1.5, 80.25),
 ('Japan', 1.5, 84.0),
 ('Portugal', 1.5, 83.25),
 ('Brazil', 1.6, 85.8),
 ('Tunisia', 1.6666666666666667, 82.33333333333

In [27]:
def average_goals_vs_ot_per_game():
    return session.query(Team.country, func.avg(Statistics.goals), func.avg(Statistics.on_target)).join(Statistics).group_by(Team.country).order_by(func.avg(Statistics.goals).asc()).all()

average_goals_vs_ot_per_game()

[('Australia', 0.6666666666666666, 2.6666666666666665),
 ('Costa Rica', 0.6666666666666666, 3.0),
 ('Egypt', 0.6666666666666666, 1.6666666666666667),
 ('Germany', 0.6666666666666666, 6.666666666666667),
 ('Iceland', 0.6666666666666666, 4.0),
 ('Iran', 0.6666666666666666, 1.3333333333333333),
 ('Morocco', 0.6666666666666666, 3.3333333333333335),
 ('Panama', 0.6666666666666666, 2.6666666666666665),
 ('Peru', 0.6666666666666666, 3.6666666666666665),
 ('Poland', 0.6666666666666666, 2.6666666666666665),
 ('Saudi Arabia', 0.6666666666666666, 3.3333333333333335),
 ('Serbia', 0.6666666666666666, 2.3333333333333335),
 ('Denmark', 0.75, 3.0),
 ('Mexico', 0.75, 3.25),
 ('Korea Republic', 1.0, 3.6666666666666665),
 ('Nigeria', 1.0, 3.0),
 ('Sweden', 1.2, 4.0),
 ('Switzerland', 1.25, 3.5),
 ('Senegal', 1.3333333333333333, 4.0),
 ('Uruguay', 1.4, 4.4),
 ('Argentina', 1.5, 4.5),
 ('Colombia', 1.5, 3.0),
 ('Japan', 1.5, 4.0),
 ('Portugal', 1.5, 3.25),
 ('Brazil', 1.6, 7.6),
 ('Tunisia', 1.666666666666

In [28]:
def average_bp_vs_pa_per_game():
    return session.query(Team.country, func.avg(Statistics.ball_possession), func.avg(Statistics.pass_accuracy)).join(Statistics).group_by(Team.country).order_by(func.avg(Statistics.ball_possession).asc()).all()

average_bp_vs_pa_per_game()

[('Iran', 32.666666666666664, 68.0),
 ('Iceland', 37.333333333333336, 68.66666666666667),
 ('Russia', 38.6, 75.4),
 ('Sweden', 38.6, 73.2),
 ('Panama', 39.0, 82.0),
 ('Costa Rica', 40.333333333333336, 76.0),
 ('Korea Republic', 40.666666666666664, 76.33333333333333),
 ('Senegal', 44.666666666666664, 78.0),
 ('Egypt', 45.0, 80.33333333333333),
 ('Serbia', 45.0, 78.33333333333333),
 ('Denmark', 45.5, 80.0),
 ('Nigeria', 45.666666666666664, 78.66666666666667),
 ('France', 47.57142857142857, 81.85714285714286),
 ('Uruguay', 48.2, 78.6),
 ('Morocco', 49.666666666666664, 82.33333333333333),
 ('Colombia', 49.75, 80.25),
 ('Poland', 50.0, 82.0),
 ('Tunisia', 50.666666666666664, 82.33333333333333),
 ('Australia', 51.333333333333336, 84.33333333333333),
 ('Peru', 51.333333333333336, 81.0),
 ('Japan', 52.25, 84.0),
 ('Belgium', 52.714285714285715, 86.0),
 ('England', 53.857142857142854, 85.42857142857143),
 ('Portugal', 54.0, 83.25),
 ('Mexico', 54.5, 83.0),
 ('Croatia', 55.285714285714285, 82.0)

In [29]:
def average_bp_vs_ot_per_game():
    return session.query(Team.country, func.avg(Statistics.ball_possession), func.avg(Statistics.on_target)).join(Statistics).group_by(Team.country).order_by(func.avg(Statistics.ball_possession).asc()).all()

average_bp_vs_ot_per_game()

[('Iran', 32.666666666666664, 1.3333333333333333),
 ('Iceland', 37.333333333333336, 4.0),
 ('Russia', 38.6, 3.8),
 ('Sweden', 38.6, 4.0),
 ('Panama', 39.0, 2.6666666666666665),
 ('Costa Rica', 40.333333333333336, 3.0),
 ('Korea Republic', 40.666666666666664, 3.6666666666666665),
 ('Senegal', 44.666666666666664, 4.0),
 ('Egypt', 45.0, 1.6666666666666667),
 ('Serbia', 45.0, 2.3333333333333335),
 ('Denmark', 45.5, 3.0),
 ('Nigeria', 45.666666666666664, 3.0),
 ('France', 47.57142857142857, 4.142857142857143),
 ('Uruguay', 48.2, 4.4),
 ('Morocco', 49.666666666666664, 3.3333333333333335),
 ('Colombia', 49.75, 3.0),
 ('Poland', 50.0, 2.6666666666666665),
 ('Tunisia', 50.666666666666664, 4.333333333333333),
 ('Australia', 51.333333333333336, 2.6666666666666665),
 ('Peru', 51.333333333333336, 3.6666666666666665),
 ('Japan', 52.25, 4.0),
 ('Belgium', 52.714285714285715, 5.714285714285714),
 ('England', 53.857142857142854, 3.5714285714285716),
 ('Portugal', 54.0, 3.25),
 ('Mexico', 54.5, 3.25),
 

In [30]:
def average_bp_vs_goals_per_game():
    return session.query(Team.country, func.avg(Statistics.ball_possession), func.avg(Statistics.goals)).join(Statistics).group_by(Team.country).order_by(func.avg(Statistics.ball_possession).asc()).all()

average_bp_vs_goals_per_game()

[('Iran', 32.666666666666664, 0.6666666666666666),
 ('Iceland', 37.333333333333336, 0.6666666666666666),
 ('Russia', 38.6, 2.2),
 ('Sweden', 38.6, 1.2),
 ('Panama', 39.0, 0.6666666666666666),
 ('Costa Rica', 40.333333333333336, 0.6666666666666666),
 ('Korea Republic', 40.666666666666664, 1.0),
 ('Senegal', 44.666666666666664, 1.3333333333333333),
 ('Egypt', 45.0, 0.6666666666666666),
 ('Serbia', 45.0, 0.6666666666666666),
 ('Denmark', 45.5, 0.75),
 ('Nigeria', 45.666666666666664, 1.0),
 ('France', 47.57142857142857, 2.0),
 ('Uruguay', 48.2, 1.4),
 ('Morocco', 49.666666666666664, 0.6666666666666666),
 ('Colombia', 49.75, 1.5),
 ('Poland', 50.0, 0.6666666666666666),
 ('Tunisia', 50.666666666666664, 1.6666666666666667),
 ('Australia', 51.333333333333336, 0.6666666666666666),
 ('Peru', 51.333333333333336, 0.6666666666666666),
 ('Japan', 52.25, 1.5),
 ('Belgium', 52.714285714285715, 2.2857142857142856),
 ('England', 53.857142857142854, 1.7142857142857142),
 ('Portugal', 54.0, 1.5),
 ('Mexic

In [31]:
def average_bp_vs_dc_per_game():
    return session.query(Team.country, func.avg(Statistics.ball_possession), func.avg(Statistics.distance_covered)).join(Statistics).group_by(Team.country).order_by(func.avg(Statistics.ball_possession).asc()).all()

average_bp_vs_dc_per_game()

[('Iran', 32.666666666666664, 99.66666666666667),
 ('Iceland', 37.333333333333336, 104.0),
 ('Russia', 38.6, 125.0),
 ('Sweden', 38.6, 104.2),
 ('Panama', 39.0, 96.33333333333333),
 ('Costa Rica', 40.333333333333336, 106.0),
 ('Korea Republic', 40.666666666666664, 105.66666666666667),
 ('Senegal', 44.666666666666664, 101.33333333333333),
 ('Egypt', 45.0, 109.33333333333333),
 ('Serbia', 45.0, 112.0),
 ('Denmark', 45.5, 115.5),
 ('Nigeria', 45.666666666666664, 99.0),
 ('France', 47.57142857142857, 99.85714285714286),
 ('Uruguay', 48.2, 103.4),
 ('Morocco', 49.666666666666664, 106.33333333333333),
 ('Colombia', 49.75, 108.75),
 ('Poland', 50.0, 98.33333333333333),
 ('Tunisia', 50.666666666666664, 103.33333333333333),
 ('Australia', 51.333333333333336, 109.66666666666667),
 ('Peru', 51.333333333333336, 101.66666666666667),
 ('Japan', 52.25, 99.0),
 ('Belgium', 52.714285714285715, 103.85714285714286),
 ('England', 53.857142857142854, 116.42857142857143),
 ('Portugal', 54.0, 100.0),
 ('Mexi

In [32]:
def average_pa_vs_goals_per_game():
    return session.query(Team.country, func.avg(Statistics.pass_accuracy), func.avg(Statistics.goals)).join(Statistics).group_by(Team.country).order_by(func.avg(Statistics.pass_accuracy).asc()).all()

average_pa_vs_goals_per_game()

[('Iran', 68.0, 0.6666666666666666),
 ('Iceland', 68.66666666666667, 0.6666666666666666),
 ('Sweden', 73.2, 1.2),
 ('Russia', 75.4, 2.2),
 ('Costa Rica', 76.0, 0.6666666666666666),
 ('Korea Republic', 76.33333333333333, 1.0),
 ('Senegal', 78.0, 1.3333333333333333),
 ('Serbia', 78.33333333333333, 0.6666666666666666),
 ('Uruguay', 78.6, 1.4),
 ('Nigeria', 78.66666666666667, 1.0),
 ('Denmark', 80.0, 0.75),
 ('Colombia', 80.25, 1.5),
 ('Egypt', 80.33333333333333, 0.6666666666666666),
 ('Peru', 81.0, 0.6666666666666666),
 ('France', 81.85714285714286, 2.0),
 ('Croatia', 82.0, 2.0),
 ('Panama', 82.0, 0.6666666666666666),
 ('Poland', 82.0, 0.6666666666666666),
 ('Morocco', 82.33333333333333, 0.6666666666666666),
 ('Tunisia', 82.33333333333333, 1.6666666666666667),
 ('Mexico', 83.0, 0.75),
 ('Portugal', 83.25, 1.5),
 ('Japan', 84.0, 1.5),
 ('Switzerland', 84.25, 1.25),
 ('Australia', 84.33333333333333, 0.6666666666666666),
 ('Argentina', 85.25, 1.5),
 ('England', 85.42857142857143, 1.714285714

In [33]:
def average_pa_vs_dc_per_game():
    return session.query(Team.country, func.avg(Statistics.pass_accuracy), func.avg(Statistics.distance_covered)).join(Statistics).group_by(Team.country).order_by(func.avg(Statistics.pass_accuracy).asc()).all()
average_pa_vs_dc_per_game()

[('Iran', 68.0, 99.66666666666667),
 ('Iceland', 68.66666666666667, 104.0),
 ('Sweden', 73.2, 104.2),
 ('Russia', 75.4, 125.0),
 ('Costa Rica', 76.0, 106.0),
 ('Korea Republic', 76.33333333333333, 105.66666666666667),
 ('Senegal', 78.0, 101.33333333333333),
 ('Serbia', 78.33333333333333, 112.0),
 ('Uruguay', 78.6, 103.4),
 ('Nigeria', 78.66666666666667, 99.0),
 ('Denmark', 80.0, 115.5),
 ('Colombia', 80.25, 108.75),
 ('Egypt', 80.33333333333333, 109.33333333333333),
 ('Peru', 81.0, 101.66666666666667),
 ('France', 81.85714285714286, 99.85714285714286),
 ('Croatia', 82.0, 116.71428571428571),
 ('Panama', 82.0, 96.33333333333333),
 ('Poland', 82.0, 98.33333333333333),
 ('Morocco', 82.33333333333333, 106.33333333333333),
 ('Tunisia', 82.33333333333333, 103.33333333333333),
 ('Mexico', 83.0, 95.25),
 ('Portugal', 83.25, 100.0),
 ('Japan', 84.0, 99.0),
 ('Switzerland', 84.25, 105.75),
 ('Australia', 84.33333333333333, 109.66666666666667),
 ('Argentina', 85.25, 99.5),
 ('England', 85.4285714

In [35]:
def average_pa_vs_ot_per_game():
    return session.query(Team.country, func.avg(Statistics.pass_accuracy), func.avg(Statistics.on_target)).join(Statistics).group_by(Team.country).order_by(func.avg(Statistics.pass_accuracy).asc()).all()

average_pa_vs_ot_per_game()

[('Iran', 68.0, 1.3333333333333333),
 ('Iceland', 68.66666666666667, 4.0),
 ('Sweden', 73.2, 4.0),
 ('Russia', 75.4, 3.8),
 ('Costa Rica', 76.0, 3.0),
 ('Korea Republic', 76.33333333333333, 3.6666666666666665),
 ('Senegal', 78.0, 4.0),
 ('Serbia', 78.33333333333333, 2.3333333333333335),
 ('Uruguay', 78.6, 4.4),
 ('Nigeria', 78.66666666666667, 3.0),
 ('Denmark', 80.0, 3.0),
 ('Colombia', 80.25, 3.0),
 ('Egypt', 80.33333333333333, 1.6666666666666667),
 ('Peru', 81.0, 3.6666666666666665),
 ('France', 81.85714285714286, 4.142857142857143),
 ('Croatia', 82.0, 4.142857142857143),
 ('Panama', 82.0, 2.6666666666666665),
 ('Poland', 82.0, 2.6666666666666665),
 ('Morocco', 82.33333333333333, 3.3333333333333335),
 ('Tunisia', 82.33333333333333, 4.333333333333333),
 ('Mexico', 83.0, 3.25),
 ('Portugal', 83.25, 3.25),
 ('Japan', 84.0, 4.0),
 ('Switzerland', 84.25, 3.5),
 ('Australia', 84.33333333333333, 2.6666666666666665),
 ('Argentina', 85.25, 4.5),
 ('England', 85.42857142857143, 3.5714285714285

In [36]:
def average_pa_vs_bp_per_game():
    return session.query(Team.country, func.avg(Statistics.pass_accuracy), func.avg(Statistics.ball_possession)).join(Statistics).group_by(Team.country).order_by(func.avg(Statistics.pass_accuracy).asc()).all()

average_pa_vs_bp_per_game()

[('Iran', 68.0, 32.666666666666664),
 ('Iceland', 68.66666666666667, 37.333333333333336),
 ('Sweden', 73.2, 38.6),
 ('Russia', 75.4, 38.6),
 ('Costa Rica', 76.0, 40.333333333333336),
 ('Korea Republic', 76.33333333333333, 40.666666666666664),
 ('Senegal', 78.0, 44.666666666666664),
 ('Serbia', 78.33333333333333, 45.0),
 ('Uruguay', 78.6, 48.2),
 ('Nigeria', 78.66666666666667, 45.666666666666664),
 ('Denmark', 80.0, 45.5),
 ('Colombia', 80.25, 49.75),
 ('Egypt', 80.33333333333333, 45.0),
 ('Peru', 81.0, 51.333333333333336),
 ('France', 81.85714285714286, 47.57142857142857),
 ('Croatia', 82.0, 55.285714285714285),
 ('Panama', 82.0, 39.0),
 ('Poland', 82.0, 50.0),
 ('Morocco', 82.33333333333333, 49.666666666666664),
 ('Tunisia', 82.33333333333333, 50.666666666666664),
 ('Mexico', 83.0, 54.5),
 ('Portugal', 83.25, 54.0),
 ('Japan', 84.0, 52.25),
 ('Switzerland', 84.25, 57.75),
 ('Australia', 84.33333333333333, 51.333333333333336),
 ('Argentina', 85.25, 63.75),
 ('England', 85.4285714285714

In [39]:
def average_ot_vs_pa_per_game():
    return session.query(Team.country, func.avg(Statistics.on_target), func.avg(Statistics.pass_accuracy)).join(Statistics).group_by(Team.country).order_by(func.avg(Statistics.on_target).asc()).all()

def average_ot_vs_bp_per_game():
    return session.query(Team.country, func.avg(Statistics.on_target), func.avg(Statistics.ball_possession)).join(Statistics).group_by(Team.country).order_by(func.avg(Statistics.on_target).asc()).all()

def average_ot_vs_dc_per_game():
    return session.query(Team.country, func.avg(Statistics.on_target), func.avg(Statistics.distance_covered)).join(Statistics).group_by(Team.country).order_by(func.avg(Statistics.on_target).asc()).all()

average_ot_vs_pa_per_game()


[('Iran', 1.3333333333333333, 99.66666666666667),
 ('Egypt', 1.6666666666666667, 109.33333333333333),
 ('Serbia', 2.3333333333333335, 112.0),
 ('Australia', 2.6666666666666665, 109.66666666666667),
 ('Panama', 2.6666666666666665, 96.33333333333333),
 ('Poland', 2.6666666666666665, 98.33333333333333),
 ('Colombia', 3.0, 108.75),
 ('Costa Rica', 3.0, 106.0),
 ('Denmark', 3.0, 115.5),
 ('Nigeria', 3.0, 99.0),
 ('Mexico', 3.25, 95.25),
 ('Portugal', 3.25, 100.0),
 ('Morocco', 3.3333333333333335, 106.33333333333333),
 ('Saudi Arabia', 3.3333333333333335, 102.0),
 ('Switzerland', 3.5, 105.75),
 ('England', 3.5714285714285716, 116.42857142857143),
 ('Korea Republic', 3.6666666666666665, 105.66666666666667),
 ('Peru', 3.6666666666666665, 101.66666666666667),
 ('Russia', 3.8, 125.0),
 ('Iceland', 4.0, 104.0),
 ('Japan', 4.0, 99.0),
 ('Senegal', 4.0, 101.33333333333333),
 ('Sweden', 4.0, 104.2),
 ('Croatia', 4.142857142857143, 116.71428571428571),
 ('France', 4.142857142857143, 99.85714285714286

In [40]:
average_ot_vs_bp_per_game()

[('Iran', 1.3333333333333333, 32.666666666666664),
 ('Egypt', 1.6666666666666667, 45.0),
 ('Serbia', 2.3333333333333335, 45.0),
 ('Australia', 2.6666666666666665, 51.333333333333336),
 ('Panama', 2.6666666666666665, 39.0),
 ('Poland', 2.6666666666666665, 50.0),
 ('Colombia', 3.0, 49.75),
 ('Costa Rica', 3.0, 40.333333333333336),
 ('Denmark', 3.0, 45.5),
 ('Nigeria', 3.0, 45.666666666666664),
 ('Mexico', 3.25, 54.5),
 ('Portugal', 3.25, 54.0),
 ('Morocco', 3.3333333333333335, 49.666666666666664),
 ('Saudi Arabia', 3.3333333333333335, 58.0),
 ('Switzerland', 3.5, 57.75),
 ('England', 3.5714285714285716, 53.857142857142854),
 ('Korea Republic', 3.6666666666666665, 40.666666666666664),
 ('Peru', 3.6666666666666665, 51.333333333333336),
 ('Russia', 3.8, 38.6),
 ('Iceland', 4.0, 37.333333333333336),
 ('Japan', 4.0, 52.25),
 ('Senegal', 4.0, 44.666666666666664),
 ('Sweden', 4.0, 38.6),
 ('Croatia', 4.142857142857143, 55.285714285714285),
 ('France', 4.142857142857143, 47.57142857142857),
 ('T

In [41]:

average_ot_vs_dc_per_game()

[('Iran', 1.3333333333333333, 99.66666666666667),
 ('Egypt', 1.6666666666666667, 109.33333333333333),
 ('Serbia', 2.3333333333333335, 112.0),
 ('Australia', 2.6666666666666665, 109.66666666666667),
 ('Panama', 2.6666666666666665, 96.33333333333333),
 ('Poland', 2.6666666666666665, 98.33333333333333),
 ('Colombia', 3.0, 108.75),
 ('Costa Rica', 3.0, 106.0),
 ('Denmark', 3.0, 115.5),
 ('Nigeria', 3.0, 99.0),
 ('Mexico', 3.25, 95.25),
 ('Portugal', 3.25, 100.0),
 ('Morocco', 3.3333333333333335, 106.33333333333333),
 ('Saudi Arabia', 3.3333333333333335, 102.0),
 ('Switzerland', 3.5, 105.75),
 ('England', 3.5714285714285716, 116.42857142857143),
 ('Korea Republic', 3.6666666666666665, 105.66666666666667),
 ('Peru', 3.6666666666666665, 101.66666666666667),
 ('Russia', 3.8, 125.0),
 ('Iceland', 4.0, 104.0),
 ('Japan', 4.0, 99.0),
 ('Senegal', 4.0, 101.33333333333333),
 ('Sweden', 4.0, 104.2),
 ('Croatia', 4.142857142857143, 116.71428571428571),
 ('France', 4.142857142857143, 99.85714285714286

In [38]:
def average_ot_vs_goals_per_game():
    return session.query(Team.country, func.avg(Statistics.on_target), func.avg(Statistics.goals)).join(Statistics).group_by(Team.country).order_by(func.avg(Statistics.on_target).asc()).all()

average_ot_vs_goals_per_game()

[('Iran', 1.3333333333333333, 0.6666666666666666),
 ('Egypt', 1.6666666666666667, 0.6666666666666666),
 ('Serbia', 2.3333333333333335, 0.6666666666666666),
 ('Australia', 2.6666666666666665, 0.6666666666666666),
 ('Panama', 2.6666666666666665, 0.6666666666666666),
 ('Poland', 2.6666666666666665, 0.6666666666666666),
 ('Colombia', 3.0, 1.5),
 ('Costa Rica', 3.0, 0.6666666666666666),
 ('Denmark', 3.0, 0.75),
 ('Nigeria', 3.0, 1.0),
 ('Mexico', 3.25, 0.75),
 ('Portugal', 3.25, 1.5),
 ('Morocco', 3.3333333333333335, 0.6666666666666666),
 ('Saudi Arabia', 3.3333333333333335, 0.6666666666666666),
 ('Switzerland', 3.5, 1.25),
 ('England', 3.5714285714285716, 1.7142857142857142),
 ('Korea Republic', 3.6666666666666665, 1.0),
 ('Peru', 3.6666666666666665, 0.6666666666666666),
 ('Russia', 3.8, 2.2),
 ('Iceland', 4.0, 0.6666666666666666),
 ('Japan', 4.0, 1.5),
 ('Senegal', 4.0, 1.3333333333333333),
 ('Sweden', 4.0, 1.2),
 ('Croatia', 4.142857142857143, 2.0),
 ('France', 4.142857142857143, 2.0),
 

In [42]:
def return_average_goals_per_game():
    return session.query(Team.country, func.avg(Statistics.goals)).join(Statistics).group_by(Team.country).all()

return_average_goals_per_game()

[('Argentina', 1.5),
 ('Australia', 0.6666666666666666),
 ('Belgium', 2.2857142857142856),
 ('Brazil', 1.6),
 ('Colombia', 1.5),
 ('Costa Rica', 0.6666666666666666),
 ('Croatia', 2.0),
 ('Denmark', 0.75),
 ('Egypt', 0.6666666666666666),
 ('England', 1.7142857142857142),
 ('France', 2.0),
 ('Germany', 0.6666666666666666),
 ('Iceland', 0.6666666666666666),
 ('Iran', 0.6666666666666666),
 ('Japan', 1.5),
 ('Korea Republic', 1.0),
 ('Mexico', 0.75),
 ('Morocco', 0.6666666666666666),
 ('Nigeria', 1.0),
 ('Panama', 0.6666666666666666),
 ('Peru', 0.6666666666666666),
 ('Poland', 0.6666666666666666),
 ('Portugal', 1.5),
 ('Russia', 2.2),
 ('Saudi Arabia', 0.6666666666666666),
 ('Senegal', 1.3333333333333333),
 ('Serbia', 0.6666666666666666),
 ('Spain', 1.75),
 ('Sweden', 1.2),
 ('Switzerland', 1.25),
 ('Tunisia', 1.6666666666666667),
 ('Uruguay', 1.4)]

In [ ]:
def return_average_bp_per_game():
    return session.query(Team.country, func.avg(Statistics.ball_possession)).join(Statistics).group_by(Team.country).all()

return_average_bp_per_game()

In [ ]:
def return_average_ot_per_game():
    return session.query(Team.country, func.avg(Statistics.on_target)).join(Statistics).group_by(Team.country).all()

return_average_ot_per_game()

In [ ]:
def return_average_pa_per_game():
    return session.query(Team.country, func.avg(Statistics.pass_accuracy)).join(Statistics).group_by(Team.country).all()

return_average_pa_per_game()

In [ ]:
def average_dc_vs_goals_per_game():
    return session.query(Team.country, func.avg(Statistics.distance_covered), func.avg(Statistics.goals)).join(Statistics).group_by(Team.country).order_by(func.avg(Statistics.distance_covered).asc()).all()

average_dc_vs_goals_per_game()

In [ ]:
def average_dc_vs_pa_per_game():
    return session.query(Team.country, func.avg(Statistics.distance_covered), func.avg(Statistics.pass_accuracy)).join(Statistics).group_by(Team.country).order_by(func.avg(Statistics.distance_covered).asc()).all()

average_dc_vs_pa_per_game()

In [ ]:
def average_dc_vs_bp_per_game():
    return session.query(Team.country, func.avg(Statistics.distance_covered), func.avg(Statistics.ball_possession)).join(Statistics).group_by(Team.country).order_by(func.avg(Statistics.distance_covered).asc()).all()

average_dc_vs_bp_per_game()

In [ ]:
def return_average_dc_per_game_desc():
    return session.query(Team.country, func.avg(Statistics.distance_covered)).group_by(Statistics.name).order_by(func.avg(Statistics.distance_covered).desc()).join(Statistics).all()

return_average_dc_per_game_desc()

In [ ]:
def top_eight_average_dc_per_game_desc():
    return session.query(Team.country, func.avg(Statistics.distance_covered)).group_by(Statistics.name).order_by(func.avg(Statistics.distance_covered).desc()).join(Statistics)[0:8]

top_eight_average_dc_per_game_desc()

In [ ]:
def return_most_dc_per_team_desc():
    return session.query(Team.country, func.sum(Statistics.distance_covered)).
group_by(Statistics.name).order_by(func.sum(Statistics.distance_covered).desc()).join(Statistics).all()

return_most_dc_per_team_desc()

In [ ]:
def return_goals_per_team_desc():
    return session.query(Team.country, func.sum(Statistics.goals)).group_by(Statistics.name).order_by(func.sum(Statistics.goals).desc()).join(Statistics).all()

return_goals_per_team_desc()

In [ ]:
def return_venue_by_goals_desc():
    return session.query(Game.venue, func.sum(Statistics.goals)).join(Statistics).group_by(Game.venue).order_by(func.sum(Statistics.goals).desc()).all()

return_venue_by_goals_desc()

In [ ]:
# #work on this later
# def return_game_winners_by_goals_desc():
#     return session.query(Game.winner, func.sum(Statistics.goals)).join(Statistics).group_by(Game.winner).order_by(func.sum(Statistics.goals).desc()).all()

# return_game_winners_by_goals_desc

In [ ]:
all_game_objects[0].teams[1].country 

In [ ]:
session.query(Statistics,func.max(Statistics.ball_possession)).join(Team)[0].Statistics.name

In [ ]:
def count_user_ids(session):
    count_userids = session.query(func.count(User.id)).one()
    return count_userids

In [ ]:
session.query(Statistics,func.max(Statistics.distance_covered)).join(Game)[0].Game.venue

In [ ]:
Team.country.query.all()

In [ ]:
def country_name_by_obj(country_object):
    for team in Team.query.all():
        if country_object == team:
            country = artist.name
            return name

In [ ]:

def list_of_countries_for_dropdown():
    options = [{'label': 'Country', 'value': 'all_countries'}]
    country_list = [country.name for country in Artist.query.all()]
    for artist in artist_list:
        options.append({'label': artist, 'value': artist})
    return options
